# MCP Registry Notebook Demo

This notebook mirrors the README flow to help you run a full MCP Registry demo end-to-end.

### What you’ll accomplish
- Publish to a local registry
- Publish to the official registry

### Who is this for?
- You’ve built (or are building) an MCP server and want to list it in a registry so clients can discover and install it.
- You prefer a safe, local dry-run before going public.


### Prerequisites
- **Docker** (for running a local registry)
- **Homebrew** (macOS/Linux) *or* Go to build from source
- **mcp-publisher** CLI

In [1]:
# Recommended install (macOS/Linux/WSL)
!brew install mcp-publisher

To reinstall 1.0.0, run:
  brew reinstall mcp-publisher


### Key concepts
- **`server.json`**: The manifest for your MCP server consumed by the registry. It must follow the published schema (see example schema below).
- **Packages vs Remotes**:
  - **Packages:** clients install and run your server locally (e.g. from npm, using `stdio`). Best for offline/local use.  
  - **Remotes:** clients connect to a hosted server you run (`sse` or `streamable-http`). Best when your server needs an always-on endpoint. 
- **Registry API**: Primary routes are `GET /v0/servers` (list/search) and `GET /v0/servers/{id}` (details).

Why this matters: the registry uses your `server.json` to index, show details, and verify install sources; clients rely on this to find and connect to your server.

## Option A - Local Registry Setup

This path runs a **local MCP Registry** and publishes a demo server entry so you can show the flow **without owning a namespace**.

In [2]:
# We will set REGISTRY once and reuse it.
REGISTRY="http://localhost:8080"

### 1. Start a local registry

Starts Docker services as defined in `docker-compose.yml`. The API will be available at `http://localhost:8080/v0` with PostgreSQL and seed data.


In [3]:
!docker compose up -d


[+] Running 0/1
 ⠋ postgres Pulling                                                        0.1s 
[+] Running 0/1
 ⠙ postgres Pulling                                                        0.2s 
[+] Running 0/1
 ⠹ postgres Pulling                                                        0.3s 
[+] Running 0/1
 ⠸ postgres Pulling                                                        0.4s 
[+] Running 0/1
 ⠼ postgres Pulling                                                        0.5s 
[+] Running 0/1
 ⠴ postgres Pulling                                                        0.6s 
[+] Running 0/1
 ⠦ postgres Pulling                                                        0.7s 
[+] Running 0/1
 ⠧ postgres Pulling                                                        0.8s 
[+] Running 0/1
 ⠇ postgres Pulling                                                        0.9s 
[+] Running 0/1
 ⠏ postgres Pulling                                                        1.0s 
[+] Running 1/1
 ✔ postgres Pu

### 2. Create and validate your `server.json`

If you don't already have a `server.json`, create one now:


In [190]:
# Generate a starter file in the current directory
!mcp-publisher init

Created server.json

Edit server.json to update:
  • Server name and description
  • Package details
  • Environment variables

Then publish with:
  mcp-publisher login github  # or your preferred auth method
  mcp-publisher publish


Or create it manually in the project root as `server.json` with at least these fields:

**Working Example** `server.json`:
```json
{
  "$schema": "https://static.modelcontextprotocol.io/schemas/2025-07-09/server.schema.json",
  "name": "io.github.<YOUR_GITHUB_USERNAME>/<your-server>",
  "description": "Short description of what your MCP server does",
  "status": "active",
  "version": "1.0.0",
  "repository": { "url": "https://github.com/<YOUR_GITHUB_USERNAME>/<your-repo>", "source": "github" },
  "remotes": [
    { "type": "streamable-http", "url": "https://<GITHUB_USERNAME>.github.io/mcp" }
  ]
}
```

- Use a namespace you control for `name` (for GitHub, `io.github.<YOUR_GITHUB_USERNAME>/<your-server>`).
- If you host a remote endpoint, set `remotes[].type` to `streamable-http` or `sse` and provide a reachable HTTPS URL.
- Increment `version` whenever you publish a new release.

Schema reference: https://registry.modelcontextprotocol.io/docs#/operations/publish-server (the `$schema` URL above is current as of this guide).

### 3. Authentication (local)

For this example, we’ll demonstrate authentication using **GitHub**.

Run the following command to login to your local registry and follow instructions:

In [5]:
!mcp-publisher login github --registry="$REGISTRY"

Logging in with github...

To authenticate, please:
1. Go to: https://github.com/login/device
2. Enter code: EA1A-DCF3
3. Authorize this application
Waiting for authorization...
Successfully authenticated!
✓ Successfully logged in


> **Note:** Local registries often don’t require login, but this example shows the full flow.  

> For the **official registry**, authentication is always required (see Option B below).


### 4. Publish MCP Server

Run the command below to publish your `server.json` to the registry:

In [6]:
!mcp-publisher publish --registry="$REGISTRY"

Publishing to http://localhost:8080...
✓ Successfully published


This will register your server entry in the registry so clients can discover it.

### 5. Search & capture the server ID

In [7]:
# Find your entry
!curl -s "$REGISTRY/v0/servers?search=demo-mcp-server" | jq '.'

{
  "servers": [
    {
      "$schema": "https://static.modelcontextprotocol.io/schemas/2025-07-09/server.schema.json",
      "name": "io.github.kevincogan/demo-mcp-server",
      "description": "Demo server entry for local testing",
      "status": "active",
      "repository": {
        "url": "https://github.com/kevincogan/demo-mcp-server",
        "source": "github"
      },
      "version": "1.0.4",
      "remotes": [
        {
          "type": "streamable-http",
          "url": "https://kevincogan.github.io/mcp"
        }
      ],
      "_meta": {
        "io.modelcontextprotocol.registry/official": {
          "id": "4d97444b-9e51-4374-abcf-ec34f46439ca",
          "published_at": "2025-09-22T13:15:05.331306252Z",
          "updated_at": "2025-09-22T13:15:05.331306252Z",
          "is_latest": true
        }
      }
    }
  ],
  "metadata": {
    "count": 1
  }
}


In [8]:
# (Optional) grab the latest ID into a shell variable
!curl -s "$REGISTRY/v0/servers?search=demo-mcp-server&version=latest" | jq -r '.servers[0]._meta["io.modelcontextprotocol.registry/official"].id'

4d97444b-9e51-4374-abcf-ec34f46439ca


The Registry supports `search`, `version=latest`, pagination, etc. (See more queries below.)

### 6. Demo a new version & “latest” behavior

Increment the version in `server.json` (manually or via the provided `jq` snippet), publish again, and show which version is latest.


In [9]:
# Increment the version in server.json (example from 1.0.0 -> 1.0.1)
# (use your editor to increment the version in the server.json”)
!mcp-publisher publish --registry="$REGISTRY"

Publishing to http://localhost:8080...
✓ Successfully published


In [10]:
# show only the latest version
!curl -s "$REGISTRY/v0/servers?search=demo-mcp-server&version=latest" | jq '.'

{
  "servers": [
    {
      "$schema": "https://static.modelcontextprotocol.io/schemas/2025-07-09/server.schema.json",
      "name": "io.github.kevincogan/demo-mcp-server",
      "description": "Demo server entry for local testing",
      "status": "active",
      "repository": {
        "url": "https://github.com/kevincogan/demo-mcp-server",
        "source": "github"
      },
      "version": "1.0.5",
      "remotes": [
        {
          "type": "streamable-http",
          "url": "https://kevincogan.github.io/mcp"
        }
      ],
      "_meta": {
        "io.modelcontextprotocol.registry/official": {
          "id": "1dfff46c-cb72-448e-9d18-61bb494737c1",
          "published_at": "2025-09-22T13:15:43.288971089Z",
          "updated_at": "2025-09-22T13:15:43.288971089Z",
          "is_latest": true
        }
      }
    }
  ],
  "metadata": {
    "count": 1
  }
}


### 7. Cleanup

Stops the Docker services and removes the local database directory.


In [11]:
# docker compose
!docker compose down

[+] Running 0/1
 ⠋ Container registry  Stopping                                            0.1s 
[+] Running 1/2
 ✔ Container registry  Removed                                             0.1s 
 ⠋ Container postgres  Stopping                                            0.0s 
[+] Running 1/2
 ✔ Container registry  Removed                                             0.1s 
 ⠙ Container postgres  Stopping                                            0.1s 
[+] Running 2/3
 ✔ Container registry            Removed                                   0.1s 
 ✔ Container postgres            Removed                                   0.2s 
 ⠋ Network mcp_registry_default  Removin...                                0.0s 
[+] Running 2/3
 ✔ Container registry            Removed                                   0.1s 
 ✔ Container postgres            Removed                                   0.2s 
 ⠙ Network mcp_registry_default  Removin...                                0.1s 
[+] Running 3/3
 ✔ Container 

In [12]:
# delete registry database
!rm -rf .db  

## Option B - Publish to the official registry

Requires GitHub OAuth and a valid `server.json`.


### 1. Install the publisher (if not already)

Uses Homebrew to install `mcp-publisher` (same command as above).


In [132]:
!brew install mcp-publisher

To reinstall 1.0.0, run:
  brew reinstall mcp-publisher


### 2. Authenticate (GitHub OAuth)

Sets the official registry URL and authenticates with GitHub.


In [150]:
# We will set REGISTRY once and reuse it.
REGISTRY="https://registry.modelcontextprotocol.io"

# Set your GitHub username
YOUR_GITHUB_USERNAME="kevincogan"

In [137]:
# Authentication via GitHub
!mcp-publisher login github --registry="$REGISTRY"

Logging in with github...

To authenticate, please:
1. Go to: https://github.com/login/device
2. Enter code: A613-26B5
3. Authorize this application
Waiting for authorization...
Successfully authenticated!
✓ Successfully logged in


This opens a device-auth flow in your browser and stores a token locally.

### 3. Create and validate your `server.json` (skip Option A if you like)

If you don't already have a `server.json`, create one now:

In [13]:
# Generate a starter file in the current directory
!mcp-publisher init

Created server.json

Edit server.json to update:
  • Server name and description
  • Package details
  • Environment variables

Then publish with:
  mcp-publisher login github  # or your preferred auth method
  mcp-publisher publish


Or create it manually in the project root as `server.json` with at least these fields:

**Example** `server.json`:
```json
{
  "$schema": "https://static.modelcontextprotocol.io/schemas/2025-07-09/server.schema.json",
  "name": "io.github.<YOUR_GITHUB_USERNAME>/<your-server>",
  "description": "Short description of what your MCP server does",
  "status": "active",
  "version": "1.0.0",
  "repository": { "url": "https://github.com/<YOUR_GITHUB_USERNAME>/<your-repo>", "source": "github" },
  "remotes": [
    { "type": "streamable-http", "url": "https://<GITHUB_USERNAME>.github.io/mcp" }
  ]
}
```

- Use a namespace you control for `name` (for GitHub, `io.github.<YOUR_GITHUB_USERNAME>/<your-server>`).
- If you host a remote endpoint, set `remotes[].type` to `streamable-http` or `sse` and provide a reachable HTTPS URL.
- Increment `version` whenever you publish a new release.

Schema reference: https://registry.modelcontextprotocol.io/docs#/operations/publish-server (the `$schema` URL above is current as of this guide).

### 4. Publish:

Publishes to the official registry.


In [141]:
!mcp-publisher publish --registry="$REGISTRY"


Publishing to https://registry.modelcontextprotocol.io...
✓ Successfully published


### 5. Verify via API

Search the registry for your server (replace the search string to match your `name`).


In [151]:
!curl -s "$REGISTRY/v0/servers?search=io.github.$YOUR_GITHUB_USERNAME/demo-mcp-server&version=latest" | jq '.'

{
  "servers": [
    {
      "$schema": "https://static.modelcontextprotocol.io/schemas/2025-07-09/server.schema.json",
      "name": "io.github.kevincogan/demo-mcp-server",
      "description": "Demo server entry for local testing",
      "status": "active",
      "repository": {
        "url": "https://github.com/kevincogan/demo-mcp-server",
        "source": "github"
      },
      "version": "1.0.4",
      "remotes": [
        {
          "type": "streamable-http",
          "url": "https://kevincogan.github.io/mcp"
        }
      ],
      "_meta": {
        "io.modelcontextprotocol.registry/official": {
          "serverId": "78162e28-0738-46be-b2ef-b1c6469a1616",
          "versionId": "91c1fc9a-833b-4186-84f1-49b79c23f0c2",
          "publishedAt": "2025-09-22T11:43:15.242712891Z",
          "updatedAt": "2025-09-22T11:43:15.242712891Z",
          "isLatest": true
        }
      }
    }
  ],
  "metadata": {
    "count": 1
  }
}


Public API supports listing and fetching server details without auth.